In [ ]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter
from collections import defaultdict
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
import urllib.request


# **1- Video source configuration and Download video from Vecteezy**

In [ ]:
VIDEO_URL = "https://www.vecteezy.com/video/20749845-traffic-jam-in-city"
VIDEO_PATH = "traffic_jam.mp4"

try:
    print(f"Downloading traffic video from {VIDEO_URL}...")
    urllib.request.urlretrieve(VIDEO_URL, VIDEO_PATH)
    print("Video downloaded successfully.")
except Exception as e:
    print(f"⚠️ Error downloading video: {e}")
    print("Please upload a traffic video manually.")
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        VIDEO_PATH = list(uploaded.keys())[0]
        print(f"Using uploaded video: {VIDEO_PATH}")
    else:
        print("⚠️ Error: No video provided. Using sample video.")
        VIDEO_PATH = "sample_traffic.mp4"

# **2- DeepSORT-inspired Tracker with Kalman Filter**

In [ ]:
class VehicleTracker:
    def __init__(self):
        self.track_id = 0
        self.tracks = {}  # Active tracks {id: bbox}
        self.kalman_filters = {}  # Kalman filters for each track
        self.trajectories = defaultdict(list)  # Movement history
        self.inactive_counts = defaultdict(int)  # Frames since last detection
        self.max_inactive = 30  # Remove tracks after this many inactive frames
        self.max_trajectory = 50  # Max points to store per trajectory
        self.vehicle_classes = {2, 5, 7}  # COCO classes: car, bus, truck

    def update(self, detections):
        # Convert detections to numpy array and filter vehicles
        dets = np.array([d for d in detections if d[5] in self.vehicle_classes])

        # Initialize updated tracks and matched indices
        updated_tracks = {}
        active_ids = set()
        matched_det_indices = set()

        # Only process if we have detections
        if len(dets) > 0:
            # Step 1: Match existing tracks with detections
            if len(self.tracks) > 0:
                # Calculate centers for distance matching
                track_centers = np.array([[(t[0]+t[2])/2, (t[1]+t[3])/2]
                                         for t in self.tracks.values()])
                det_centers = np.array([[(d[0]+d[2])/2, (d[1]+d[3])/2]
                                      for d in dets])

                # Compute cost matrix and solve assignment problem
                cost_matrix = distance_matrix(track_centers, det_centers)
                row_ind, col_ind = linear_sum_assignment(cost_matrix)

                # Process matches
                for i, j in zip(row_ind, col_ind):
                    if cost_matrix[i, j] < 100:  # Maximum allowed distance
                        tid = list(self.tracks.keys())[i]
                        updated_tracks[tid] = dets[j][:4]
                        active_ids.add(tid)
                        matched_det_indices.add(j)

                        # Update Kalman filter
                        center = [(dets[j][0]+dets[j][2])/2, (dets[j][1]+dets[j][3])/2]
                        kf = self.kalman_filters[tid]
                        kf.predict()
                        kf.update(np.array(center))

                        # Update trajectory
                        self.trajectories[tid].append(center)
                        if len(self.trajectories[tid]) > self.max_trajectory:
                            self.trajectories[tid].pop(0)

                        self.inactive_counts[tid] = 0

            # Step 2: Create new tracks for unmatched detections
            for j in range(len(dets)):
                if j not in matched_det_indices:
                    self.track_id += 1
                    tid = self.track_id
                    updated_tracks[tid] = dets[j][:4]
                    active_ids.add(tid)

                    # Initialize Kalman filter
                    center = [(dets[j][0]+dets[j][2])/2, (dets[j][1]+dets[j][3])/2]
                    kf = KalmanFilter(dim_x=4, dim_z=2)
                    kf.F = np.array([[1,0,1,0], [0,1,0,1], [0,0,1,0], [0,0,0,1]])  # State transition
                    kf.H = np.array([[1,0,0,0], [0,1,0,0]])  # Measurement function
                    kf.P *= 1000  # Covariance matrix
                    kf.R = np.array([[5,0], [0,5]])  # Measurement noise
                    kf.Q = np.eye(4) * 0.01  # Process noise
                    kf.x = np.array([center[0], center[1], 0, 0])  # Initial state

                    self.kalman_filters[tid] = kf
                    self.trajectories[tid].append(center)
                    self.inactive_counts[tid] = 0

        # Update tracks and remove inactive ones
        self.tracks = updated_tracks
        self._remove_inactive_tracks()

        return self.tracks, self.kalman_filters, self.trajectories

    def _remove_inactive_tracks(self):
        for tid in list(self.tracks.keys()):
            self.inactive_counts[tid] += 1
            if self.inactive_counts[tid] > self.max_inactive:
                del self.tracks[tid]
                del self.kalman_filters[tid]
                del self.trajectories[tid]
                del self.inactive_counts[tid]

# **3- Initialize YOLOv8 model and tracker**

In [ ]:
model = YOLO('yolov8n.pt')  # Using nano version for Colab compatibility
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError("Cannot open video file")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
if fps <= 0:
    fps = 30  # Default FPS if not detected

tracker = VehicleTracker()  # Initialize tracker without frame_height

# **4- Output video writer**

In [ ]:
output_path = "tracking_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# **5- Visualization settings**

In [ ]:
COLORS = {
    'car': (0, 255, 0),      # Green
    'bus': (0, 165, 255),    # Orange
    'truck': (0, 0, 255),    # Red
    'trajectory': (255, 255, 0),  # Yellow
    'prediction': (255, 0, 255)   # Purple
}

CLASS_NAMES = {
    2: 'car',
    5: 'bus',
    7: 'truck'
}

# **6-Main processing loop**

In [ ]:
frame_count = 0
try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        print(f"Processing frame {frame_count}...")

        # Run YOLO detection
        results = model(frame, conf=0.3, verbose=False)  # Lower confidence for busy traffic

        # Prepare detections [x1, y1, x2, y2, conf, class]
        detections = []
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = box.conf.item()
                cls = box.cls.item()
                detections.append([x1, y1, x2, y2, conf, cls])

        # Update tracker
        tracks, kfs, trajectories = tracker.update(detections)

        # Visualization
        for tid, bbox in tracks.items():
            x1, y1, x2, y2 = map(int, bbox)

            # Get vehicle class (use last detection's class)
            class_id = None
            for det in detections:
                if (abs((det[0]+det[2])/2 - (x1+x2)/2) < 10 and
                    abs((det[1]+det[3])/2 - (y1+y2)/2) < 10):
                    class_id = int(det[5])
                    break

            color = COLORS.get(CLASS_NAMES.get(class_id, 'car'), (0, 255, 0))

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{CLASS_NAMES.get(class_id, 'vehicle')} {tid}",
                        (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Draw trajectory
            if tid in trajectories:
                trajectory = trajectories[tid]
                for i in range(1, len(trajectory)):
                    cv2.line(frame,
                            (int(trajectory[i-1][0]), int(trajectory[i-1][1])),
                            (int(trajectory[i][0]), int(trajectory[i][1])),
                            COLORS['trajectory'], 2)

            # Draw Kalman filter prediction
            if tid in kfs:
                kf = kfs[tid]
                pred_x, pred_y = int(kf.x[0]), int(kf.x[1])
                cv2.circle(frame, (pred_x, pred_y), 5, COLORS['prediction'], -1)
                vel_x, vel_y = int(kf.x[2]), int(kf.x[3])
                cv2.arrowedLine(frame, (pred_x, pred_y),
                               (pred_x + vel_x*5, pred_y + vel_y*5),
                               COLORS['prediction'], 2, tipLength=0.3)

        # Display vehicle count
        cv2.putText(frame, f"Vehicles: {len(tracks)}", (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Write frame to output
        out.write(frame)

except Exception as e:
    print(f"Error during processing: {e}")
finally:
    cap.release()
    out.release()
    print("Processing completed. Saving output video...")

# Download the output video
if os.path.exists(output_path):
    print(f"Output video saved to {output_path}")
    from google.colab import files
    files.download(output_path)
else:
    print("Error: Output video not created")